In [1]:

import dash
import dash_html_components as html
from dash.dependencies import Input, Output
from markupsafe import Markup
from dash import dcc
import plotly.express as px
import pandas as pd
import numpy as np
from faker import Faker
import plotly.graph_objs as go
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.utilities import JsCode
from folium.features import GeoJsonPopup
from folium.plugins import TimeSliderChoropleth
from folium import plugins
from folium.plugins import HeatMap



/var/folders/c5/7b2hdd417y90r2mfyk9z39mw0000gn/T/ipykernel_97440/1066862009.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
#Create 

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 15000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7,9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17,22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

#Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))


aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement


# Define buffer distances
buffer_distances = {
    'outside': 200,  # ~200 meters in degrees
    'inside': 150,   # ~150 meters in degrees
    'difficult': 125  # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Initialize a dictionary to hold all the new columns
new_columns_dict = {}

# Pre-calculate colors based on all possible combinations of days and hours
#Tuesday, opens: 13,  closes: 17
#Tuesdaz, hour = 9
    #true, false, true > false: red
#tuesday 14
    #true , true, true 
for day in daysofweek_categories:
    for hour in range(24):
        column_name = f'color_{day}_{hour}'
        condition = (
            (practice_df['daysofweek'].apply(lambda x: day in x)) & 
            (practice_df['opening_hour'] <= hour) & 
            (practice_df['closing_hour'] >= hour)
        )
        new_columns_dict[column_name] = np.where(condition, 'green', 'red')

# Create a DataFrame from the dictionary
color_df = pd.DataFrame(new_columns_dict)

# Concatenate the new columns to the original DataFrame in a single operation
practice_df = pd.concat([practice_df, color_df], axis=1)

practice_df.head()


# Convert DataFrame to GeoDataFrame
practice_gdf = gpd.GeoDataFrame(
    practice_df,
    geometry=[Point(xy) for xy in zip(practice_df['Longitude'], practice_df['Latitude'])],
    crs="EPSG:4326"  # WGS84 coordinate system
)


 
  

def generate_folium_map(selected_day, selected_hour):
    color_column = f'color_{selected_day}_{selected_hour}'
    practice_gdf['color'] = practice_df[color_column]

    m = folium.Map(location=[50.5, 4.3517], zoom_start=8)
    for idx, row in practice_gdf.iterrows():
        folium.Circle(
            location=(row['Latitude'], row['Longitude']),
            radius=row['buffer_distance'],
            color=row ['color'],
            fill=True,
            fill_opacity=row['opacity'],
            stroke=False
        ).add_to(m)
    return m.get_root().render()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id='day-radioitems',
        options=[{'label': day, 'value': day} for day in daysofweek_categories],
        value='Monday',
        inline=True
    ),
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    html.Iframe(
        id='folium-map',
        srcDoc=generate_folium_map('Monday', 12),
        width='100%',
        height='600px'
    )
])

@app.callback(
    Output('folium-map', 'srcDoc'),
    [Input('day-radioitems', 'value'), Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    return generate_folium_map(selected_day, selected_hour)

if __name__ == '__main__':
    app.run_server(debug=True)

In [3]:
practice_df.head()

,daysofweek,opening_hour,closing_hour,opacity,Latitude,Longitude,aed_coordinates,aed_placement,buffer_distance,color_Monday_0,...,color_Sunday_16,color_Sunday_17,color_Sunday_18,color_Sunday_19,color_Sunday_20,color_Sunday_21,color_Sunday_22,color_Sunday_23,geometry,color
0,[Sunday],7,17,0.763139,49.931571,4.542425,"(49.93157130733513, 4.542425321018182)",inside,150,red,...,green,green,red,red,red,red,red,red,POINT (4.54243 49.93157),red
1,"[Thursday, Wednesday, Monday, Friday]",8,20,0.615854,50.690467,4.715315,"(50.69046678991123, 4.715314640932323)",outside,200,red,...,red,red,red,red,red,red,red,red,POINT (4.71531 50.69047),green
2,[Saturday],8,18,0.121289,50.605940,5.857030,"(50.60593963666725, 5.857029568510243)",difficult,125,red,...,red,red,red,red,red,red,red,red,POINT (5.85703 50.60594),red
3,"[Tuesday, Monday, Sunday]",7,17,0.797758,50.218796,5.870685,"(50.21879610610471, 5.870684917307507)",outside,200,red,...,green,green,red,red,red,red,red,red,POINT (5.87068 50.21880),green
4,[Sunday],8,19,0.114180,50.609178,5.911255,"(50.609178439915546, 5.911255048526388)",outside,200,red,...,green,green,green,green,red,red,red,red,POINT (5.91126 50.60918),red
